In [173]:
# Basic Libraries

import numpy as np
import scipy as sp
import sympy as smp
from scipy.integrate import solve_ivp, odeint
import itertools
import mpmath
import opt_einsum as oe
import numba as nb
import symengine as sme
from scipy.spatial import KDTree
from scipy.sparse import coo_array

# For Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# For Debugging and Profiling
import snoop
from cProfile import Profile
from pstats import SortKey, Stats
import pyperclip

In [174]:
def get_norm(T, g):
    """
    T should have confuguration either 'u' or 'uu
    Returned Configuration: Scalar
    """
    if T.ndim == 2:
        return np.einsum("ua,vb,ab,uv->", g, g, T, T)

    else:
        return np.einsum("ab,a,b->", g, T, T)

In [175]:
def levi_civita_symbol(dim=4):
    """
    Define Levi Cevita Tensor
    Config: uuuu
    """

    arr = np.zeros(tuple([dim for _ in range(dim)]))
    for x in itertools.permutations(tuple(range(dim))):
        mat = np.zeros((dim, dim), dtype=np.int32)
        for i, j in zip(range(dim), x):
            mat[i, j] = 1
        arr[x] = int(np.linalg.det(mat))
    return arr

In [176]:
# @snoop
def levi_civita_tensor(gk):
    #! uuuu
    epsu = levi_civita_symbol()
    det_g = smp.det(smp.Matrix(gk))
    return epsu / sme.sqrt(-(det_g))

In [177]:
def cov_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: ll
    """
    #! Correction Added Mass term in the symbols
    g = np.zeros((4, 4), dtype=type(r))

    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    g[0, 0] = -(1.0 - 2.0 * M * r / Sigma)
    g[1, 1] = Sigma / Delta
    g[2, 2] = Sigma
    g[3, 3] = (
        sme.sin(theta) ** 2
        * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
        / Sigma
    )
    g[0, 3] = -2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma
    g[3, 0] = g[0, 3]

    # return g.astype(float)
    return g

In [178]:
def cot_metric_tensor(r, theta, a, M=1.0):
    """
    Define the metric tensor function.

    Parameters:
        a (float): Parameter 'a' in the metric.
        r (float): Parameter 'r' in the metric.
        theta (float): Parameter 'theta' in the metric.

    Returns:
        np.ndarray: The metric tensor at the given values of a, r, and theta.
        Configuration: uu
    """
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    denom = Delta * sme.sin(theta) ** 2

    g = np.zeros((4, 4), dtype=type(r))
    g[0, 0] = (
        -(
            sme.sin(theta) ** 2
            * ((r**2 + a**2) ** 2 - Delta * a**2 * sme.sin(theta) ** 2)
            / Sigma
        )
        / denom
    )

    g[1, 1] = Delta / Sigma

    g[2, 2] = 1.0 / Sigma

    # g[3, 3] = -(-(1.0 - 2.0 * r / Sigma)) / denom
    #! Corrected the g[3,3] term below
    g[3, 3] = (Delta - a**2 * sme.sin(theta) ** 2) / (denom * Sigma)

    g[0, 3] = (-2.0 * a * M * r * sme.sin(theta) ** 2 / Sigma) / denom

    g[3, 0] = g[0, 3]

    return g
    # return g

In [179]:
def kerr_christoffel(r, theta, a, M=1.0):
    """
    Function to give the christoffel symbols for kerr metric.
    The christoffel symbols are given as \Gamma ^i _{jk}

    From Reference Paper, Appendix
    Config: ull
    """

    cs = np.zeros((4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    cs[3, 0, 1] = M * (2 * r**2 - Sigma) / (Delta * Sigma**2) * a
    cs[0, 0, 1] = (
        M * (r**2 + a**2) * (r**2 - a**2 * sme.cos(theta) ** 2) / (Sigma**2 * Delta)
    )

    cs[3, 0, 2] = -2 * M * a * r * (sme.cos(theta) / sme.sin(theta)) / (Sigma**2)
    cs[0, 0, 2] = -2 * M * a**2 * r * sme.sin(theta) * sme.cos(theta) / Sigma**2

    cs[0, 1, 3] = (
        -M
        * a
        * (2 * r**2 * (r**2 + a**2) + Sigma * (r**2 - a**2))
        * sme.sin(theta) ** 2
        / (Delta * Sigma**2)
    )

    cs[3, 1, 3] = (
        r * Sigma * (Sigma - 2 * M * r)
        - M * a**2 * (2 * r**2 - Sigma) * sme.sin(theta) ** 2
    ) / (Delta * Sigma**2)

    cs[0, 2, 3] = M * a**3 * r * sme.sin(theta) ** 2 * sme.sin(2 * theta) / Sigma**2

    cs[3, 2, 3] = (
        (sme.cos(theta) / sme.sin(theta))
        * (Sigma**2 + 2 * M * a**2 * r * sme.sin(theta) ** 2)
        / Sigma**2
    )

    cs[1, 0, 0] = M * Delta * (2 * r**2 - Sigma) / Sigma**3

    cs[1, 0, 3] = -cs[1, 0, 0] * a * sme.sin(theta) ** 2

    cs[2, 0, 0] = -M * a * r * sme.sin(2 * theta) / Sigma**3 * a
    cs[2, 0, 3] = (
        2.0 * M * r * a * (r**2 + a**2) * sme.sin(theta) * sme.cos(theta) / Sigma**3
    )

    cs[1, 1, 1] = r / Sigma + (M - r) / Delta

    cs[1, 2, 2] = -r * Delta / Sigma
    cs[2, 1, 2] = -cs[1, 2, 2] / Delta

    cs[1, 1, 2] = cs[2, 2, 2] = -(a**2) * sme.sin(2 * theta) / (2 * Sigma)
    cs[2, 1, 1] = -cs[1, 1, 2] / Delta

    cs[1, 3, 3] = (
        -Delta
        * (r * Sigma**2 - M * a**2 * (2 * r**2 - Sigma) * sme.sin(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**3
    )

    cs[2, 3, 3] = (
        -(Delta * Sigma**2 + 2 * M * r * (r**2 + a**2) ** 2)
        * sme.sin(2 * theta)
        / (2 * Sigma**3)
    )

    cs[0, 1, 0] = cs[0, 0, 1]
    cs[0, 2, 0] = cs[0, 0, 2]
    cs[0, 3, 1] = cs[0, 1, 3]
    cs[0, 3, 2] = cs[0, 2, 3]

    cs[1, 3, 0] = cs[1, 0, 3]
    cs[1, 2, 1] = cs[1, 1, 2]

    cs[2, 3, 0] = cs[2, 0, 3]
    cs[2, 2, 1] = cs[2, 1, 2]

    cs[3, 1, 0] = cs[3, 0, 1]
    cs[3, 2, 0] = cs[3, 0, 2]
    cs[3, 3, 1] = cs[3, 1, 3]
    cs[3, 3, 2] = cs[3, 2, 3]

    # return symmetrize(arr=cs)+np.zeros((4,4,4))
    return cs + np.zeros((4, 4, 4))
    # return cs

In [180]:
def kerr_riemann_tensor(r, theta, a, M=1.0, config="ulll"):
    """
    Define variables

    Components of the Riemann tensor for Kerr Metric
    From Reference Paper, Appendix
    The Configuration is ulll
    """

    rijkl = np.zeros((4, 4, 4, 4), dtype=type(r))

    # Definitions
    Delta = a**2 - 2.0 * M * r + r**2
    Sigma = r**2 + a**2 * sme.cos(theta) ** 2

    rijkl[0, 0, 0, 3] = (
        2.0
        * M**2
        * sme.sin(theta) ** 2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**4
    )

    rijkl[0, 0, 1, 2] = (
        -2.0
        * M**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * r
        * sme.cos(theta)
        * a**2
        * sme.sin(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 1] = (
        M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 0, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 1, 3] = (
        3.0
        * M
        * sme.sin(theta) ** 2
        * a
        * r
        * (r**2 + a**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / (Sigma**3 * Delta)
    )

    rijkl[0, 1, 2, 3] = (
        -M
        * sme.cos(theta)
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * sme.sin(theta) ** 2 * Delta)
        / (Sigma**3 * Delta)
    )

    # Antisymmetric under exchange of last two indices.
    rijkl[0, 0, 3, 0] = -rijkl[0, 0, 0, 3]
    rijkl[0, 0, 2, 1] = -rijkl[0, 0, 1, 2]
    rijkl[0, 1, 1, 0] = -rijkl[0, 1, 0, 1]
    rijkl[0, 1, 2, 0] = -rijkl[0, 1, 0, 2]
    rijkl[0, 1, 3, 1] = -rijkl[0, 1, 1, 3]
    rijkl[0, 1, 3, 2] = -rijkl[0, 1, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[0, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2 * a**2 * sme.sin(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 2, 1, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * ((a**2 + r**2) ** 2 + 2.0 * a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )
    rijkl[0, 2, 2, 3] = (
        -3.0
        * M
        * sme.sin(theta) ** 2
        * r
        * a
        * (a**2 + r**2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[0, 3, 0, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 + 3 * a**2 * sme.sin(theta) ** 2 - 3 * a**2)
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        / Sigma**4
    )
    rijkl[0, 3, 1, 2] = (
        (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * M
        * ((a**2 + r**2) ** 2 - a**2 * sme.sin(theta) ** 2 * Delta)
        * a
        * sme.sin(theta)
        * sme.cos(theta)
        / (Sigma**3 * Delta)
    )

    # Symmetries
    rijkl[0, 2, 1, 0] = -rijkl[0, 2, 0, 1]
    rijkl[0, 2, 2, 0] = -rijkl[0, 2, 0, 2]
    rijkl[0, 2, 3, 1] = -rijkl[0, 2, 1, 3]
    rijkl[0, 2, 3, 2] = -rijkl[0, 2, 2, 3]
    rijkl[0, 3, 3, 0] = -rijkl[0, 3, 0, 3]
    rijkl[0, 3, 2, 1] = -rijkl[0, 3, 1, 2]

    #! ------------------------------------------------------------------------------#

    rijkl[1, 0, 0, 1] = (
        r
        * M
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        / Sigma**4
    )
    rijkl[1, 0, 0, 2] = (
        -3.0
        * M
        * Delta
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[1, 0, 1, 3] = (
        r
        * M
        * a
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[1, 0, 2, 3] = (
        -a
        * M
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 2, 0, 3] = (
        -sme.cos(theta)
        * M
        * sme.sin(theta)
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        / Sigma**3
    )
    rijkl[1, 2, 1, 2] = -M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**2
    rijkl[1, 3, 0, 1] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        / Sigma**4
    )
    rijkl[1, 3, 0, 2] = (
        a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[1, 3, 1, 3] = (
        -sme.sin(theta) ** 2
        * M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * ((r**2 + a**2) ** 2 + 2.0 * a**2 * Delta * sme.sin(theta) ** 2)
        / Sigma**4
    )
    rijkl[1, 3, 2, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * Delta
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )

    # Symmetries
    rijkl[1, 0, 1, 0] = -rijkl[1, 0, 0, 1]
    rijkl[1, 0, 2, 0] = -rijkl[1, 0, 0, 2]
    rijkl[1, 0, 3, 1] = -rijkl[1, 0, 1, 3]
    rijkl[1, 0, 3, 2] = -rijkl[1, 0, 2, 3]
    rijkl[1, 2, 3, 0] = -rijkl[1, 2, 0, 3]
    rijkl[1, 2, 2, 1] = -rijkl[1, 2, 1, 2]
    rijkl[1, 3, 1, 0] = -rijkl[1, 3, 0, 1]
    rijkl[1, 3, 2, 0] = -rijkl[1, 3, 0, 2]
    rijkl[1, 3, 3, 1] = -rijkl[1, 3, 1, 3]
    rijkl[1, 3, 3, 2] = -rijkl[1, 3, 2, 3]

    #! -------------------------------------------------------------------------------------#

    rijkl[2, 0, 0, 1] = (
        -3.0
        * M
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta)
        * sme.cos(theta)
        / Sigma**4
    )
    rijkl[2, 0, 0, 2] = (
        -M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        / Sigma**4
    )
    rijkl[2, 0, 1, 3] = (
        -a
        * M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 0, 2, 3] = (
        -M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 1, 0, 3] = (
        M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**3
    )
    rijkl[2, 1, 1, 2] = (
        M * r * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**2)
    )
    rijkl[2, 3, 0, 1] = (
        M
        * a
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / Sigma**4
    )
    rijkl[2, 3, 0, 2] = (
        M
        * r
        * a
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[2, 3, 1, 3] = (
        3.0
        * M
        * a**2
        * (r**2 + a**2)
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta) ** 3
        / Sigma**4
    )
    rijkl[2, 3, 2, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (2.0 * (r**2 + a**2) ** 2 + a**2 * Delta * sme.sin(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )

    # Symmetries
    rijkl[2, 0, 1, 0] = -rijkl[2, 0, 0, 1]
    rijkl[2, 0, 2, 0] = -rijkl[2, 0, 0, 2]
    rijkl[2, 0, 3, 1] = -rijkl[2, 0, 1, 3]
    rijkl[2, 0, 3, 2] = -rijkl[2, 0, 2, 3]
    rijkl[2, 1, 3, 0] = -rijkl[2, 1, 0, 3]
    rijkl[2, 1, 2, 1] = -rijkl[2, 1, 1, 2]
    rijkl[2, 3, 1, 0] = -rijkl[2, 3, 0, 1]
    rijkl[2, 3, 2, 0] = -rijkl[2, 3, 0, 2]
    # rijkl[2, 3, 2, 1] = -rijkl[2, 3, 1, 2]
    rijkl[2, 3, 3, 1] = -rijkl[2, 3, 1, 3]
    rijkl[2, 3, 3, 2] = -rijkl[2, 3, 2, 3]

    #! --------------------------------------------------------------------------#

    rijkl[3, 0, 1, 2] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 - Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 1] = (
        3.0 * M * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / (Delta * Sigma**3)
    )
    rijkl[3, 1, 0, 2] = (
        -M
        * a
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (2.0 * a**2 * sme.sin(theta) ** 2 + Delta)
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 1, 3] = (
        M
        * r
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * (r**2 + a**2 + 2.0 * a**2 * sme.sin(theta) ** 2)
        / (Delta * Sigma**3)
    )
    rijkl[3, 1, 2, 3] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 2.0 * M * r)
        * a**2
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 1] = (
        -M
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (a**2 * sme.sin(theta) ** 2 + 2.0 * Delta)
        * a
        * (sme.cos(theta) / sme.sin(theta))
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 0, 2] = (
        -M * 3.0 * r * a * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2) / Sigma**3
    )
    rijkl[3, 2, 1, 3] = (
        -M
        * a**2
        * (3 * r**2 - a**2 * sme.cos(theta) ** 2)
        * (3.0 * (r**2 + a**2) - 4.0 * M * r)
        * sme.sin(theta)
        * sme.cos(theta)
        / (Delta * Sigma**3)
    )
    rijkl[3, 2, 2, 3] = (
        -M
        * r
        * (2.0 * (r**2 + a**2) + a**2 * sme.sin(theta) ** 2)
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        / Sigma**3
    )
    rijkl[3, 3, 0, 3] = (
        -2.0
        * M**2
        * a
        * r**2
        * (r**2 - 3.0 * a**2 * sme.cos(theta) ** 2)
        * sme.sin(theta) ** 2
        / Sigma**4
    )
    rijkl[3, 3, 1, 2] = (
        2.0
        * M**2
        * r
        * a**2
        * (3.0 * r**2 - a**2 * sme.cos(theta) ** 2)
        * sme.cos(theta)
        * sme.sin(theta)
        / (Delta * Sigma**3)
    )

    # Symmetries
    rijkl[3, 0, 2, 1] = -rijkl[3, 0, 1, 2]
    rijkl[3, 1, 1, 0] = -rijkl[3, 1, 0, 1]
    rijkl[3, 1, 2, 0] = -rijkl[3, 1, 0, 2]
    rijkl[3, 1, 3, 1] = -rijkl[3, 1, 1, 3]
    rijkl[3, 1, 3, 2] = -rijkl[3, 1, 2, 3]
    rijkl[3, 2, 1, 0] = -rijkl[3, 2, 0, 1]
    rijkl[3, 2, 2, 0] = -rijkl[3, 2, 0, 2]
    rijkl[3, 2, 3, 1] = -rijkl[3, 2, 1, 3]
    rijkl[3, 2, 3, 2] = -rijkl[3, 2, 2, 3]
    rijkl[3, 3, 3, 0] = -rijkl[3, 3, 0, 3]
    rijkl[3, 3, 2, 1] = -rijkl[3, 3, 1, 2]

    #!---------------------------------------------------------------------#

    part1 = a**2 + 2 * r * (-2 * M + r) + a**2 * sme.cos(2 * theta)
    part2 = 3 * a**2 - 2 * r**2 + 3 * a**2 * sme.cos(2 * theta)
    part3 = (a**2 + 2 * r**2 + a**2 * sme.cos(2 * theta)) ** 4
    # Calculating Rtensor elements

    # rijkl[3, 0, 0, 3] =  (part1 + part2 + part3 + part4) / (Sigma**5)
    rijkl[3, 0, 0, 3] = 4 * M * r * part1 * part2 / part3
    rijkl[3, 0, 3, 0] = -rijkl[3, 0, 0, 3]

    # return rijkl
    if config == "ulll":
        return rijkl + np.zeros((4, 4, 4, 4))

    elif config == "llll":
        gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
        return np.einsum("ij,jklm->iklm", gk, rijkl) + np.zeros((4, 4, 4, 4))

    # elif config=="lluu":
    #     #!Config: lluu
    #     gkinv = cot_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     gk = cov_metric_tensor(r=r, theta=theta, a=a, M=M)
    #     rllll = np.einsum("ij,jklm->iklm", gk, rijkl)

    #     return np.einsum("kjlm,la,mb->kjab", rllll, gkinv, gkinv)

    else:
        return "Invalid String"

In [181]:
def spintensor(levi, pvector, svector, m):

    # p_vectorl = np.einsum("ij,j->i", gk, pvector)  #! Lower the pvector
    # s_vectorl = np.einsum("ij,j->i", gk, svector)  #! Lower the svector

    #! Using above definition
    return -np.einsum("uvab,a,b->uv", levi, svector, pvector) / m

In [182]:
def metric_D(r, theta, a, M=1.0):
    dgkt = np.zeros((4, 4), dtype=type(r))
    dgkp = np.zeros((4, 4), dtype=type(r))

    denom = (r**2 + a**2 * sme.cos(theta) ** 2) ** 2

    dgkt[1, 0] = M * (a**2 - 2 * r**2 + a**2 * sme.cos(2 * theta)) / denom
    dgkt[1, 3] = (
        -(2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2)
        / denom
    )
    dgkt[2, 0] = 2 * a**2 * M * r * sme.sin(2 * theta) / denom
    dgkt[2, 3] = -2 * a * M * r * (r**2 + a**2) * sme.sin(2 * theta) / denom

    dgkp[1, 0] = (
        -2 * a * M * (-(r**2) + a**2 * sme.cos(theta) ** 2) * sme.sin(theta) ** 2 / denom
    )

    dgkp[1, 3] = (
        2
        * sme.sin(theta) ** 2
        * (
            sme.cos(theta) ** 2
            * (2 * a**2 * r * (a**2 + r**2) + a**4 * (M - r) * sme.sin(theta) ** 2)
            + r * (-(a**4) + r**4 + a**2 * (a**2 - M * r) * sme.sin(theta) ** 2)
        )
        / denom
    )

    dgkp[2, 0] = -2 * a * M * r * (a**2 + r**2) * sme.sin(2 * theta) / denom
    dgkp[2, 3] = (
        (
            3 * a**6
            + 10 * a**4 * M * r
            + 11 * a**4 * r**2
            + 16 * a**2 * M * r**3
            + 16 * a**2 * r**4
            + 8 * r**6
            + 4
            * a**2
            * (a**2 + 2 * r**2)
            * (a**2 + r * (-2 * M + r))
            * sme.cos(2 * theta)
            + a**4 * (a**2 - 2 * M * r + r**2) * sme.cos(4 * theta)
        )
        * sme.sin(2 * theta)
        / (8 * denom)
    )

    return dgkt.T + np.zeros((4, 4)), dgkp.T + np.zeros((4, 4))

In [183]:
# @snoop
def energy(pvector, Stensor, dgkt):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkt, Stensor) / 2.0
    return -pvector[0] + term2

In [184]:
def angular_momentum(pvector, Stensor, dgkp):
    #! Changed Here
    # pvecl = np.einsum("ij,j->i", gk, pvector)
    term2 = np.einsum("uv,uv->", dgkp, Stensor) / 2.0
    return pvector[3] - term2

In [185]:
def calculate_four_momentum(pvector, uvector, svector, cs, dRlluu):
    # correction = np.einsum(
    #     "abps,psuv,b,u,v->a", Riemann, levi_mixed, uvector, svector, pvector
    # ) / (2.0 * m)
    correction = -np.einsum(
        "uvab,v,a,b->u", dRlluu, uvector, pvector, svector, optimize=True
    )
    dp = correction + np.einsum("auv,a,u->v", cs, pvector, uvector)

    return dp

In [186]:
def calculate_four_spin(pvector, uvector, svector, cs, dRuluu):

    # correction= -np.einsum("u,abps,psgd,a,b,g,d->u",pvector,Riemann,svector,uvector,pvector,svector,optimize=True)/2.0
    correction = -np.einsum(
        "u,abgd,a,b,g,d->u",
        pvector,
        dRuluu,
        svector,
        uvector,
        pvector,
        svector,
        optimize=True,
    )

    ds = correction + np.einsum("auv,a,u->v", cs, svector, uvector)

    return ds

In [187]:
# @snoop
def calculate_four_velocity(gk, gkinv, pvector, svector, ddRuuuu,return_N=False):
    # scalar_divisor = np.einsum("uvps,uv,ps->", Riemann_cov, Stensor, Stensor) / 4.0

    # correction = np.einsum(
    #     "ab,bguv,g,uv->a", Stensor, Riemann_cov, pvector, Stensor
    # ) / (2.0 * (m**2 + scalar_divisor))
    correction = -np.einsum("uabg,a,b,g->u", ddRuuuu, svector, pvector, svector)
    pvecu = np.einsum("ij,j->i", gkinv, pvector)

    dx = pvecu + correction
    # Vsq = np.einsum("uv,u,v->", gk, dx, dx)

    # Pv = sme.sqrt(-1 / Vsq)
    Pv = np.sqrt(
        1.0 / (1.0 - gk.dot(correction).dot(correction) + 2.0 * pvector.dot(correction))
    )

    # Pv = 1.0
    dx = dx * Pv

    if return_N:
        return dx,Pv
    return dx

In [188]:
# @snoop
def MPD(t, y, p):
    t, r, theta, phi, pt, pr, ptheta, pphi, st, sr, stheta, sphi = y

    a, m, M = p

    # xvector=y[:4]
    pvector = y[4:8]  #! p_u
    svector = y[8:]  #! s_u

    gk = cov_metric_tensor(r, theta, a, M)  #! g_{ij}
    gkinv = cot_metric_tensor(r, theta, a, M)
    cs = kerr_christoffel(r, theta, a, M)  #! G^i_{jk}

    Riemann = kerr_riemann_tensor(r, theta, a, M, "ulll")  #! R^a_{bcd}

    # Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
    # Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

    levi = levi_civita_tensor(gk)  #! e^{abcd}
    # levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
    levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

    dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi)
    dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

    ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed)

    # stensor = spintensor(levi, pvector=pvector, svector=svector, gk=gk, m=m)  #! S^{uv}

    uvector = calculate_four_velocity(
        gk=gk, gkinv=gkinv, pvector=pvector, svector=svector, ddRuuuu=ddRuuuu
    )  #!dx_mu

    dp = calculate_four_momentum(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRlluu=dRlluu
    )
    #!dp_{mu}

    ds = calculate_four_spin(
        pvector=pvector, uvector=uvector, svector=svector, cs=cs, dRuluu=dRuluu
    )
    #!ds^{mu}

    dy = np.zeros_like(y)

    dy[0:4] = uvector
    dy[4:8] = dp
    dy[8:] = ds

    return dy

In [189]:
t, r, theta, phi = smp.symbols("t r theta phi")
pt,pr,ptheta,pphi=sme.symbols("pt pr ptheta pphi")
st, sr, stheta, sphi = sme.symbols("st sr stheta sphi")
a,m,M= sme.symbols("a m M")

In [190]:
gk= cov_metric_tensor(r, theta,a ,M)

In [191]:
# t, r, theta, phi, pt, pr, ptheta, pphi, st, sr, stheta, sphi=[y(i) for i in range(12)]

In [192]:
p=np.array([a,m,M])

In [193]:
y0=np.array([t,r,theta,phi,pt,pr,ptheta,pphi,st,sr,stheta,sphi])

In [194]:
M_ = 1e0
m_ = 1e0

S = 1e0 * m_ * M_  # 1e0 * m_ * M_
E = 0.9328 * m_
Jz = 2.8 * m_ * M_

r0 = 12.23 * M_
theta0 = np.pi / 7.0
phi0 = 0.0
a0 = 1.0 * M_

In [195]:
# v_ = np.sqrt(M_ / r0)
# abar = a0 / M_

# Epro = (1 - 2 * v_**2 + abar * v_**3) / (np.sqrt(1 - 3 * v_**2 + 2 * abar * v_**3))
# Eret = (1 - 2 * v_**2 - abar * v_**3) / (np.sqrt(1 - 3 * v_**2 - 2 * abar * v_**3))

# ideg = np.pi / 30
# ecc = 0.01
# alpha_ip = 1 + np.cos(ideg)
# alpha_im = 1 - np.cos(ideg)

# E = 0.5 * (alpha_ip * Epro + alpha_im * Eret)
# Jz = np.cos(ideg) * np.sqrt((1 - ecc**2) / (2 * (1 - E)))

In [196]:
gk = cov_metric_tensor(r=r0, theta=theta0, a=a0, M=M_)
gkinv = cot_metric_tensor(r=r0, theta=theta0, a=a0, M=M_)

In [197]:
cs = kerr_christoffel(r=r0, theta=theta0, a=a0, M=M_)

In [198]:
P1 = 1e-2 * m_

In [199]:
S1 = 1e-4 * m_
S2 = 1e-4 * m_

In [200]:
p0, p2, p3 = sme.symbols("p0 p2 p3", real=True)

In [201]:
s0, s3 = sme.symbols("s0 s3", real=True)

In [202]:
gpcoordu = np.array([p0, P1, p2, p3])
gscoordu = np.array([s0, S1, S2, s3])

In [203]:
# Definitions
Delta = a0**2 - 2 * M_ * r0 + r0**2
Sigma = r0**2 + a0**2 * sme.cos(theta0) ** 2

In [204]:
levi = levi_civita_tensor(gk)

In [205]:
# Sab = spi(sa=gsi, pb=gpi, gk=eta, epsilon=epsl)
Suv = spintensor(levi=levi, pvector=gpcoordu, svector=gscoordu, m=m_)
#!Config: uu, In Coordinate Basis

In [206]:
dgkt, dgkp = metric_D(r0, theta0, a0, M_)

In [207]:
eq210a = energy(gpcoordu, Suv, dgkt) - E

In [208]:
eq210b = angular_momentum(gpcoordu, Suv, dgkp) - Jz

In [209]:
eq533 = get_norm(T=gpcoordu, g=gkinv) + m_**2

In [210]:
eq229 = np.einsum("ij,j,i->", gkinv, gpcoordu, gscoordu) - 0.0

In [211]:
eq228 = np.einsum("ij,j,i->", gkinv, gscoordu, gscoordu) - S**2

In [212]:
def solve_eq(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return f(p_0, p_2, p_3, s_0, s_3)

In [213]:
def solve_eq_sp(p):
    f = sme.lambdify([p0, p2, p3, s0, s3], [eq210a, eq210b, eq533, eq228, eq229])
    return np.array(f(p[0], p[1], p[2], p[3], p[4]))

In [214]:
def jac_nl_symb():
    X = smp.Matrix([eq210a, eq210b, eq533, eq228, eq229])
    Y = smp.Matrix([p0, p2, p3, s0, s3])

    return X.jacobian(Y)

In [215]:
jac = jac_nl_symb()

In [216]:
def jac_nl(p_0, p_2, p_3, s_0, s_3):
    f = smp.lambdify([p0, p2, p3, s0, s3], jac)

    return f(p_0, p_2, p_3, s_0, s_3)

In [217]:
solve_eq(-0.924284, -1.33974, 2.55554, 0.0000381344, -0.000652199)

[-0.008516151961313057,
 -0.24454205497915282,
 0.22824767546796887,
 -0.9999999782175489,
 -1.764820810652289e-05]

In [218]:
roots = mpmath.findroot(
    solve_eq,
    (-0.92, -1.33, 2.55, 0.0000038, -0.00065),
    solver="mdnewton",
    tol=1e-19,
    J=jac_nl,
    verify=True,
)

ValueError: Could not find root within given tolerance. (0.0522611261397916484733 > 1e-19)
Try another starting point or tweak arguments.

In [ ]:
roots = np.array(roots.tolist(), dtype=np.float64)[:, 0]

In [58]:
solve_eq(*roots)

[-1.1102230246251565e-16,
 -4.440892098500626e-15,
 1.8540724511240114e-13,
 0.0,
 6.979593837022798e-17]

In [59]:
np0, np2, np3, ns0, ns3 = roots

In [60]:
ngpcoordu = np.array([np0, P1, np2, np3]).astype(np.float64)
ngscoordu = np.array([ns0, S1, S2, ns3]).astype(np.float64)

In [61]:
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

In [62]:
rulll = kerr_riemann_tensor(r=r0, theta=theta0, a=a0, M=M_, config="ulll")

In [63]:
dRuluu = 0.5 * np.einsum("abps,psuv->abuv", rulll, levi)
ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed)

In [64]:
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

In [65]:
vmu0 = calculate_four_velocity(
    gk=gk, gkinv=gkinv, pvector=ngpcoordu, svector=ngscoordu, ddRuuuu=ddRuuuu
)

In [66]:
#!We pass r0^mu,p^mu and S^mu
Vy0 = np.array(
    [
        0.0,
        r0,
        theta0,
        phi0,
        ngpcoordu[0],
        ngpcoordu[1],
        ngpcoordu[2],
        ngpcoordu[3],
        ngscoordu[0],
        ngscoordu[1],
        ngscoordu[2],
        ngscoordu[3],
    ]
).astype(np.float64)

In [67]:
f=MPD(0.,y0,p)

In [129]:
Vy0 = np.array([
    161.52164841145472,
    6.087951858542188,
    1.1298042147358822,
    9.282392215837014,
    -0.9344438982449085,
    -0.02439080056116108,
    -0.5953280361905549,
    2.6946583071136145,
    0.07919493360681484,
    -1.0656437568406378,
    -2.252369195025744,
    -1.6755780805836944,
])

In [130]:
Vy0.tolist()

[161.52164841145472,
 6.087951858542188,
 1.1298042147358822,
 9.282392215837014,
 -0.9344438982449085,
 -0.02439080056116108,
 -0.5953280361905549,
 2.6946583071136145,
 0.07919493360681484,
 -1.0656437568406378,
 -2.252369195025744,
 -1.6755780805836944]

In [133]:
Vp=[a0,m_,M_]

In [134]:
Vp=[2.3,1.0,1.0]

In [135]:
Vf = [
    f[i].subs(
        {
            t: Vy0[0],
            r: Vy0[1],
            theta: Vy0[2],
            phi: Vy0[3],
            pt: Vy0[4],
            pr: Vy0[5],
            ptheta: Vy0[6],
            pphi: Vy0[7],
            st: Vy0[8],
            sr: Vy0[9],
            stheta: Vy0[10],
            sphi: Vy0[11],
            a: Vp[0],
            M: Vp[2],
        }
    )
    for i in range(len(f))
]

In [136]:
np.array(Vf) - np.array(
    [
        1.1493191511726109,
        0.008389561579624175,
        -0.002139050568792576,
        0.0009599428852153357,
        9.825088322941853e-07,
        -4.626315209536312e-05,
        0.006661267438494629,
        -0.0008534236380036949,
        -0.000460614618953672,
        -0.004912052433092574,
        -1.429528824926082,
        3.49938296849447,
    ]
)

array([0.182521947203695, -0.0283346153194189, -0.0136761490615742,
       0.0987802313399659, 0.000615152970256947, 0.0212974582448888,
       0.0938586059634684, -0.0090181948878729, -0.0232281133033001,
       -0.0147123550974135, 1.29009125384342, -2.941477898723],
      dtype=object)

In [137]:
jacob = np.array([sme.Derivative(f[i], j) for i in range(len(f)) for j in y0])

In [138]:
jacobr=jacob.reshape((12,12))

In [139]:
Vjacob = [jacob[i].subs(
    {
        t: Vy0[0],
        r: Vy0[1],
        theta: Vy0[2],
        phi: Vy0[3],
        pt: Vy0[4],
        pr: Vy0[5],
        ptheta: Vy0[6],
        pphi: Vy0[7],
        st: Vy0[8],
        sr: Vy0[9],
        stheta: Vy0[10],
        sphi: Vy0[11],
        a: Vp[0],
        M: Vp[2],
    }
) for i in range(len(jacob))]

In [140]:
Vjacobr= np.array(Vjacob).reshape((12,12)).astype(float)

In [141]:
Njacob = np.array([[0.0, -0.0793013149681327, -0.1175359542863243, 0.0, 0.3354670959189461, -0.02695770261176843, -0.021636733703851486, 0.1113078811952205, -0.000773158215306346, 0.0034399857748643042, 0.0003715680550306961, 1.1692344296064798e-05], [0.0, 0.0002131886063202003, -0.00014789052367737828, 0.0, -0.02703972296947422, 0.8153798773430057, 0.0005168378030457405, -0.0018821117770278192, 0.00851051077959747, -0.00023093380241584856, 4.84105446379068e-05, 0.0003714853285848786], [0.0, 0.005151258342561303, -0.0002367425656815993, 0.0, -0.0221364383538638, 0.000523347189021454, 0.027203227292384786, -0.0016616767320330225, -0.0005211311064213517, -0.000562906169279756, 1.294367177323131e-05, -0.00015033040721061232], [0.0, -0.03320230744318949, -0.08437637610622961, 0.0, 0.11205367389852841, -0.001887138015358201, -0.001633110435037038, 0.038479686400883596, -0.0003058969696384695, 0.0010764066352929042, 0.00013297100191735135, -2.5233354115699817e-05], [0.0, -0.0005086434931642118, -0.0002578891069778569, 0.0, -3.47284206382897e-05, -0.0019290869635158695, -0.0005253367245770384, 8.308439766318437e-05, -0.0007891579932688931, -0.0003811532353580082, 2.8170307127694638e-05, -0.00019117186360334039], [0.0, -0.016057777875390627, -0.04380075979752405, 0.0, 0.07451251378368817, -1.705371828459794e-05, -0.0049416250506657084, 0.032633694156273034, 0.0004364834628560198, -0.003499264197705824, 0.0004114666483934599, 0.00022768220265605786], [0.0, -0.029219852029771228, -0.36562663276101887, 0.0, 0.1003588166762618, -0.0002738021460555228, 0.00022323342351569206, 0.0721522931605123, -0.002923250665169114, 0.011589000498106563, 0.002220362687570782, -0.00011219957303303766], [0.0, 0.005407180026288976, -0.0024975978550598535, 0.0, 0.002530428507053269, 0.0019421737321682273, 0.011446057725962622, -0.00023956462514524107, 0.010715502653126973, 0.0055361116997694765, 9.429701258249351e-05, 0.0024335114064995738], [0.0, 0.007505301305377239, -0.00815598665316494, 0.0, -0.011945740323102215, 0.00016695638928718705, 0.0016451515208472422, -0.002891184069629858, 0.0007833744167822079, 0.017896864518605157, 0.0010119559442190872, 0.0006552310330481673], [0.0, 0.009519114356396192, 0.022488149179480048, 0.0, -0.020456445986824816, 0.0031479791366357196, -0.007828843634290216, -0.00877180150224319, 0.02827764740356504, 0.000807074141816548, -0.002582430043488541, 0.015923812828064397], [0.0, 0.030386321945952872, 0.22326947592483037, 0.0, -0.1991284929905596, -0.24344381014898436, 0.14349196249471186, -0.038990389292097055, -0.023989883229253026, 0.07074528822115805, -0.0018916585109329258, 0.03608490250890602], [0.0, -0.10795372593203423, -0.08371053097622404, 0.0, 0.6282371913124023, -0.20531834764795925, -0.03374465929256458, 0.20598825836127105, -0.003471655155824728, -0.41696023670234855, -0.03635528497559072, -0.012087349144343243]])

In [150]:
np.abs(Vjacobr - Njacob).max()

2.220446049250313e-16

In [148]:
print((Vjacobr-Njacob)-1e-16)

[[-1.00000000e-16 -7.22444244e-17  1.10223025e-17 -1.00000000e-16
  -1.00000000e-16 -9.30611061e-17 -1.13877788e-16 -1.55511151e-16
  -1.00433681e-16 -1.10408341e-16 -9.98915798e-17 -9.98373697e-17]
 [-1.00000000e-16 -9.80755411e-17 -1.00000000e-16 -1.00000000e-16
  -9.30611061e-17 -1.00000000e-16 -1.00000000e-16 -9.95663191e-17
  -9.82652765e-17 -9.98644747e-17 -1.00013553e-16 -1.00162630e-16]
 [-1.00000000e-16 -9.82652765e-17 -9.91326383e-17 -1.00000000e-16
  -9.65305530e-17 -1.00108420e-16 -1.00000000e-16 -9.93494787e-17
  -9.92410585e-17 -9.97831596e-17 -1.00015247e-16 -9.99186848e-17]
 [-1.00000000e-16 -1.13877788e-16 -1.13877788e-16 -1.00000000e-16
  -1.00000000e-16 -9.91326383e-17 -1.00650521e-16 -1.06938894e-16
  -1.00054210e-16 -1.01517883e-16 -9.99728949e-17 -9.99830593e-17]
 [-1.00000000e-16 -1.00000000e-16 -1.00216840e-16 -1.00000000e-16
  -1.00189735e-16 -9.93494787e-17 -9.96747393e-17 -1.00027105e-16
  -9.96747393e-17 -9.97831596e-17 -1.00003388e-16 -9.98915798e-17]
 [-1.

In [88]:
gk = cov_metric_tensor(r, theta, a, M)  #! g_{ij}
gkinv = cot_metric_tensor(r, theta, a, M)
cs = kerr_christoffel(r, theta, a, M)  #! G^i_{jk}

Riemann = kerr_riemann_tensor(r, theta, a, M, "ulll")  #! R^a_{bcd}

# Riemann_cov = np.einsum("ul,lvps->uvps", gk, Riemann)  #! R_{abcd}
# Riemann_uull = np.einsum("ap,upgd->uagd", gkinv, Riemann)  #! R^{ua}_{gd}

levi = levi_civita_tensor(gk)  #! e^{abcd}
# levi_mixed = np.einsum("psxy,xu,yv->psuv", levi, gk, gk)  #! e^{ps}_{uv}
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)  #! e_{ps}^{uv}

dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi) #!R*^a_{buv}
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)  #!R*^_{ab}^{uv}

ddRuuuu = 0.5 * np.einsum("bi,aips,psuv->abuv", gkinv, dRuluu, levi_mixed) #! *R*^{abuv}


In [89]:
DdRuluu= np.zeros((4,4,4,4,4),object)
VDdRuluu= np.zeros((4,4,4,4,4))

In [90]:
vari=[t,r,theta,phi]

In [109]:
Vy0 = [
    161.52164841145472,
    6.087951858542188,
    1.1298042147358822,
    9.282392215837014,
    -0.9344438982449085,
    -0.02439080056116108,
    -0.5953280361905549,
    2.6946583071136145,
    0.07919493360681484,
    -1.0656437568406378,
    -2.252369195025744,
    -1.6755780805836944,
]

In [112]:
Vp=[2.3,1.0,1.0]

In [113]:
for x_ in range(4):
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    if dRuluu[i,j,k,l]!=0:
                        DdRuluu[i,j,k,l,x_]= dRuluu[i,j,k,l].diff(vari[x_])
                        VDdRuluu[i,j,k,l,x_]= DdRuluu[i,j,k,l,x_].subs(
                                                {
                                                    t: Vy0[0],
                                                    r: Vy0[1],
                                                    theta: Vy0[2],
                                                    phi: Vy0[3],
                                                    pt: Vy0[4],
                                                    pr: Vy0[5],
                                                    ptheta: Vy0[6],
                                                    pphi: Vy0[7],
                                                    st: Vy0[8],
                                                    sr: Vy0[9],
                                                    stheta: Vy0[10],
                                                    sphi: Vy0[11],
                                                    a: Vp[0],
                                                    M: Vp[2],
                                                }
                                            ) 

In [114]:
(VDdRuluu-np.array([[[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 5.389811557453544e-05, 7.933751014322442e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -6.07005303971133e-05, 9.060748040809436e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 6.07005303971133e-05, -9.060748040809436e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -5.389811557453544e-05, -7.933751014322442e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.004590497397524643, 0.010322203582491071, 0.0], [0.0, 0.0007056558415110554, -0.0010978955967828425, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.004590497397524643, -0.010322203582491071, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0004467920003181481, -0.0007485749390263579, 0.0]], [[0.0, -0.0007056558415110554, 0.0010978955967828425, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.00026741208183677896, 0.00011749205475456229, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0004467920003181481, 0.0007485749390263579, 0.0], [0.0, 0.00026741208183677896, -0.00011749205475456229, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.012428749797049691, -0.03313147441568294, 0.0], [0.0, -0.0026140680597889323, -0.005398882542940062, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.012428749797049691, 0.03313147441568294, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0029261679766275843, 0.002672400756743029, 0.0]], [[0.0, 0.0026140680597889323, 0.005398882542940062, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.00039289388474361274, 0.0006692374288831335, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0029261679766275843, -0.002672400756743029, 0.0], [0.0, -0.00039289388474361274, -0.0006692374288831335, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0019764293377357114, -0.004923321039551012, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0019154435782326841, -0.00508974006483269, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0019154435782326841, 0.00508974006483269, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0019764293377357114, 0.004923321039551012, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0024089550237554364, 0.00548002539976934, 0.0], [0.0, 0.00042682710898192126, -0.0007643032547457699, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0024089550237554364, -0.00548002539976934, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.000264197935874919, -0.00043769845316094555, 0.0]], [[0.0, -0.00042682710898192126, 0.0007643032547457699, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.00014328765092592075, 9.106426292381932e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.000264197935874919, 0.00043769845316094555, 0.0], [0.0, 0.00014328765092592075, -9.106426292381932e-05, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.001840741086560821, -0.0008731868360604324, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0009560041539161922, 0.002579936195672379, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0009560041539161922, -0.002579936195672379, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.001840741086560821, 0.0008731868360604324, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, -0.006329365924155957, -0.005895430824422893, 0.0], [0.0, -0.0026041143668114543, 0.007231686051490218, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.006329365924155957, 0.005895430824422893, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.001452950869839245, 0.0029000892040969605, 0.0]], [[0.0, 0.0026041143668114543, -0.007231686051490218, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0004268271089819213, -0.0007643032547457699, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.001452950869839245, -0.0029000892040969605, 0.0], [0.0, -0.0004268271089819213, 0.0007643032547457699, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0004875276393790347, -0.0008549107351538644, 0.0], [0.0, -7.155319013552367e-05, -9.610182533706838e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0004875276393790347, 0.0008549107351538644, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -9.19268954298736e-05, 7.120544612267795e-05, 0.0]], [[0.0, 7.155319013552367e-05, 9.610182533706838e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 1.1921213055008794e-05, 1.4504250502555179e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 9.19268954298736e-05, -7.120544612267795e-05, 0.0], [0.0, -1.1921213055008794e-05, -1.4504250502555179e-05, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.00010774447075057532, 2.8935264710878678e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -4.912982161870688e-05, -8.549274184636364e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 4.912982161870688e-05, 8.549274184636364e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.00010774447075057532, -2.8935264710878678e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, -0.00451955794504414, 0.012321426116322904, 0.0], [0.0, 0.0003194117682517481, 0.0001953600815364835, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.00451955794504414, -0.012321426116322904, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0004875276393790347, -0.0008549107351538644, 0.0]], [[0.0, -0.0003194117682517481, -0.0001953600815364835, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.00012068301175423053, -0.00018159456718343207, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -0.0004875276393790347, 0.0008549107351538644, 0.0], [0.0, 0.00012068301175423053, 0.00018159456718343207, 0.0], [0.0, 0.0, 0.0, 0.0]]]], [[[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -4.8403522855625505e-05, -0.00014928571817829117, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 5.136075549604711e-05, -1.985881680114134e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -5.136075549604711e-05, 1.985881680114134e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 4.8403522855625505e-05, 0.00014928571817829117, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0004467920003181481, 0.0007485749390263579, 0.0], [0.0, 0.00015966761108620367, -8.855679004368356e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.0004467920003181481, -0.0007485749390263579, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -8.341747256219e-05, -0.00019424573368992298, 0.0]], [[0.0, -0.00015966761108620367, 8.855679004368356e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -2.7604542018828537e-05, 7.951776254960512e-06, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 8.341747256219e-05, 0.00019424573368992298, 0.0], [0.0, 2.7604542018828537e-05, -7.951776254960512e-06, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.004766909063188406, -0.003545587592803463, 0.0], [0.0, -0.00039289388474361274, -0.0006692374288831335, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, -0.004766909063188406, 0.003545587592803463, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -0.0005770589375283738, 0.0002399627727103172, 0.0]], [[0.0, 0.00039289388474361274, 0.0006692374288831335, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0001318209954178155, 0.0003435314518682142, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0005770589375283738, -0.0002399627727103172, 0.0], [0.0, -0.0001318209954178155, -0.0003435314518682142, 0.0], [0.0, 0.0, 0.0, 0.0]]], [[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -5.389811557453544e-05, -7.933751014322442e-05, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 6.07005303971133e-05, -9.060748040809436e-05, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0], [0.0, -6.07005303971133e-05, 9.060748040809436e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]], [[0.0, 5.389811557453544e-05, 7.933751014322442e-05, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]]]])).max()

1.3877787807814457e-17

In [ ]:
!jupytext --output MPDJacobian.py MPDJacobian.ipynb

In [87]:
1.0 + (-8.88178420e-20)

1.0